In [1]:
import cv2
import numpy as np
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt 
from torch.utils.data import Dataset, random_split



In [2]:
class ImageDataset(Dataset):
    def __init__(self, imagesDir, label, transform=None):
        self.imagesDir = imagesDir
        self.transform = transform
        self.label = label
        _, _, files = next(os.walk(self.imagesDir))
        self.files = files
        label_map = {"AVM": 0, "Normal": 1, "Ulcer": 2}
        self.label = label_map[self.label]  # Converts "AVM" to 0, etc.

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = cv2.imread(os.path.join(self.imagesDir, self.files[idx]))
        height, width = image.shape[:2]
        mask = np.zeros((height, width), dtype=np.uint8)
        center = (width // 2, height // 2)
        radius = min(center[0], center[1]) + 17
        cv2.circle(mask, center, radius, 255, -1)
        masked = cv2.bitwise_and(image, image, mask=mask)
        
        white_bg = np.ones_like(image, dtype=np.uint8) * 255
        output = np.where(mask[:, :, np.newaxis] == 255, masked, white_bg)
        output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
        if self.transform:
            output = self.transform(output)

        return output, self.label

In [3]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet means
                         [0.229, 0.224, 0.225])  # ImageNet stds
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
image = val_transform(image)
AVMImages = ImageDataset(imagesDir='/kaggle/input/capsule-endoscopy-dataset-kauhc/AVM', label="AVM", transform=train_transform)
NormalImages = ImageDataset(imagesDir='/kaggle/input/capsule-endoscopy-dataset-kauhc/Normal', label="Normal", transform=train_transform)
UlcerImages = ImageDataset(imagesDir='/kaggle/input/capsule-endoscopy-dataset-kauhc/Ulcer', label="Ulcer", transform=train_transform)
dataset = torch.utils.data.ConcatDataset([AVMImages, NormalImages, UlcerImages])
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
validation_size = dataset_size - train_size
train_data, val_data = random_split(dataset, [train_size, validation_size])

trainLoader = DataLoader(train_data, batch_size=64, shuffle=True)
valLoader = DataLoader(val_data, batch_size=64, shuffle=True)



NameError: name 'image' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.efficientnet_b0(pretrained=True)

# Replace classifier head
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 3)  # 3 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 30
patience = 5  # for early stopping
best_val_acc = 0
counter = 0
best_model_wts = copy.deepcopy(model.state_dict())
model_save_path = "best_model.pth"

In [ ]:
device

In [ ]:
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0

    for inputs, labels in trainLoader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels)

    epoch_loss = running_loss / len(trainLoader.dataset)
    epoch_acc = correct.double() / len(trainLoader.dataset)
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    # Validation phase
    model.eval()
    val_correct = 0
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valLoader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels)

    val_loss /= len(valLoader.dataset)
    val_acc = val_correct.double() / len(valLoader.dataset)
    print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    # Check for improvement
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save("model.pth")
        torch.save(best_model_wts, model_save_path)
        print(f"✅ Saved new best model (val acc: {val_acc:.4f})")
        counter = 0  # reset early stopping
    else:
        counter += 1
        print(f"⚠️ No improvement. Early stop counter: {counter}/{patience}")
        if counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

# Load best model after training
model.load_state_dict(best_model_wts)
print(f"✅ Loaded best model weights from {model_save_path}")


In [ ]:
import cv2
import numpy as np

def ellipse_to_polygon(center, axes, angle, num_points=50):
    contour = cv2.ellipse2Poly(center=center, axes=(int(axes[0] / 2), int(axes[1] / 2)),
                               angle=int(angle), arcStart=0, arcEnd=360, delta=360 // num_points)
    return contour  # Returns Nx2 numpy array of (x, y) points

def normalize_points(points, img_width, img_height):
    return [(x / img_width, y / img_height) for (x, y) in points]

def save_yolo_segmentation(file_path, class_id, points):
    """Saves normalized polygon points in YOLOv8 segmentation format."""
    with open(file_path, 'w') as f:
        f.write(f"{class_id} " + " ".join(f"{x:.6f} {y:.6f}" for x, y in points))

def ObtainEllipse(path):
    image = cv2.imread(path)
    height, width = image.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)
    center = (width // 2, height // 2)
    radius = min(center[0], center[1]) + 17
    cv2.circle(mask, center, radius, 255, -1)
    masked = cv2.bitwise_and(image, image, mask=mask)
    white_bg = np.ones_like(image, dtype=np.uint8) * 255
    output = np.where(mask[:, :, np.newaxis] == 255, masked, white_bg)
    gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([0, 0, 0])  # Allow slightly off-black

    # Create a binary mask where pixels are close to black
    black_mask = cv2.inRange(output, lower_black, upper_black)

    contours, _ = cv2.findContours(black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if len(contour) >= 5:  # fitEllipse requires at least 5 points
            ellipse = cv2.fitEllipse(contour)
            (x_center, y_center), (MA, ma), angle = ellipse
            return ellipse, output, (x_center, y_center), (MA, ma), angle, width, height

In [ ]:
import os
import glob
import random
import shutil
import csv

# paths
src_root = "/kaggle/input/capsule-endoscopy-dataset-kauhc"
dst_root = "dataset/images"
dst_root_labels = "dataset/labels"

train_dir = os.path.join(dst_root, "train")
test_dir  = os.path.join(dst_root, "test")

labels_train_dir = os.path.join(dst_root, "train")
labels_test_dir  = os.path.join(dst_root, "test")

# make sure output dirs exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir,  exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_test_dir,  exist_ok=True)

# only these subfolders will be processed
CLASSES = {
    "AVM":   0,
    "Ulcer": 1
}

# collect all images and associate labels, skipping everything else (e.g., 'Normal')
items = []
for class_name, label in CLASSES.items():
    folder = os.path.join(src_root, class_name)
    if not os.path.isdir(folder):
        continue
    for filepath in glob.glob(os.path.join(folder, "*")):
        ext = os.path.splitext(filepath)[1].lower()
        if ext in [".jpg", ".jpeg", ".png", ".bmp"]:
            items.append((filepath, label, ext))

# shuffle and split 80% train / 20% test
random.seed(42)
random.shuffle(items)
split_idx    = int(len(items) * 0.8)
train_items  = items[:split_idx]
test_items   = items[split_idx:]

def process_list(item_list, out_dir, csv_path, start_index=1):
    """Copy & rename images, write CSV of (filename,label)."""
    with open(csv_path, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["filename", "label"])
        idx = start_index
        for src_path, label, ext in item_list:
            new_name = f"image{idx}{ext}"
            dst_path  = os.path.join(out_dir, new_name)
            shutil.copy(src_path, dst_path)
            ellipse, output, (x_center, y_center), (MA, ma), angle, width, height = ObtainEllipse("test image.bmp")
            if(MA > 5 and ma > 5):
                polygon = ellipse_to_polygon((int(x_center), int(y_center)), (MA, ma), angle)
                normalized = normalize_points(polygon, width, height)
                save_yolo_segmentation("label.txt", class_id=0, points=normalized)
            writer.writerow([new_name, label])
            idx += 1
    return idx

# execute
next_idx = process_list(
    train_items,
    train_dir,
    os.path.join(dst_root, "train_labels.csv"),
    start_index=1
)
process_list(
    test_items,
    test_dir,
    os.path.join(dst_root, "test_labels.csv"),
    start_index=next_idx
)

print(f"Done! {len(train_items)} train images and {len(test_items)} test images created.")
